#### In This Notebook We Explore How The Mams-For-ABSA is Doing Sentiment Analysis And Try To Find out The Output Variable For this Model

    The Entry Code of Traning The Model Is

In [1]:
import yaml
import os
# Import Custom Training Class From The Absa Dataset
from train.train import train

# Load The Configuration From The Config YAML File
config = yaml.safe_load(open('config.yml'))

# Check The Mode There is 2 Options for the mode Term Based And Category Based
mode = config['mode']

# Enable GPU For Traning The Model 
# This Code Forces The Pytorch Library To Use only 1 GPU 
# As Many systems have multiple gpus including my laptop
os.environ["CUDA_VISIBLE_DEVICES"] = str(config['aspect_' + mode + '_model'][config['aspect_' + mode + '_model']['type']]['gpu'])
# This Basically Pulls Out The GPU Value From The Config Value

print(os.environ["CUDA_VISIBLE_DEVICES"])

#The Main Traning Code
## train(config) -> This is The Main Function Explained Later

0


    Now If We Look Into This Train Function

In [3]:
import torch
from torch import nn
from torch import optim
from train import make_aspect_term_model, make_aspect_category_model
from train.make_data import make_term_data, make_category_data
from train.make_optimizer import make_optimizer
from train.eval import eval
import os
import time
import pickle
from src.module.utils.loss import CapsuleLoss

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

def train(config):
    # Check Which Mode The Model's Is in
    mode = config['mode']
    # Depending on the model type, it creates the model and loads the training and validation data.
    if mode == 'term':
        # The make_model and make_term_data function is explored later
        model = make_aspect_term_model.make_model(config)
        train_loader, val_loader = make_term_data(config)
    else:
        # The makde_model and the make_category_data is explored later
        model = make_aspect_category_model.make_model(config)
        train_loader, val_loader = make_category_data(config)
    # The model is moved to the GPU using model.cuda().
    model = model.cuda()
    
    base_path = config['base_path']
    # The model's path is set using model_path = os.path.join(base_path, 'checkpoints/%s.pth' % config['aspect_' + mode + '_model']['type']).
    model_path = os.path.join(base_path, 'checkpoints/%s.pth' % config['aspect_' + mode + '_model']['type'])
    
    if not os.path.exists(os.path.dirname(model_path)):
        os.makedirs(os.path.dirname(model_path))
    # The index2word dictionary is loaded from disk using with
    with open(os.path.join(base_path, 'processed/index2word.pickle'), 'rb') as handle:
        index2word = pickle.load(handle)
    
    # The CapsuleLoss function is instantiated using criterion = CapsuleLoss().
    criterion = CapsuleLoss()
    # An optimizer is created for the model using optimizer = make_optimizer(config, model).
    optimizer = make_optimizer(config, model)
    # The max_val_accuracy and min_val_loss variables are set to 0 and 100, respectively.
    max_val_accuracy = 0
    min_val_loss = 100
    global_step = 0
    config = config['aspect_' + mode + '_model'][config['aspect_' + mode + '_model']['type']]
    # A loop is run for config['aspect_' + mode + '_model'][config['aspect_' + mode + '_model']['type']]['num_epoches'] epochs. 
    # For each epoch, the training data is looped over and the model is trained using 
    # stochastic gradient descent. 
    # After each epoch, the model is evaluated on the validation data. 
    # The training and validation loss and accuracy are printed.
    for epoch in range(config['num_epoches']):
        # Initialize the total loss for the epoch to 0.
        # Initialize the total number of samples seen in the epoch to 0.
        # Initialize the total number of correct predictions in the epoch to 0.
        total_loss = 0
        total_samples = 0
        correct_samples = 0
        start = time.time() #  Start a timer to measure the time taken for the epoch.
        # This loop iterates over the batches in the training data.
        for i, data in enumerate(train_loader):
            #  Increment the global step count by 1.
            global_step += 1
            # Set the model to training mode.
            model.train()
            # Extract the inputs and labels for the current batch.
            input0, input1, label = data
            # Move the inputs and labels to the GPU if available.
            input0, input1, label = input0.cuda(), input1.cuda(), label.cuda()
            # Reset the gradients to zero.
            optimizer.zero_grad()
            #  Forward pass the inputs through the model to get the predictions.
            logit = model(input0, input1)
            # Calculate the loss based on the predictions and labels.
            loss = criterion(logit, label)
            # Get the size of the current batch.
            batch_size = input0.size(0)
            # Update the total loss for the epoch.
            total_loss += batch_size * loss.item()
            #  Update the total number of samples seen in the epoch.
            total_samples += batch_size
            # Get the predicted labels based on the highest predicted probability.
            pred = logit.argmax(dim=1)
            # Update the total number of correct predictions in the epoch.
            correct_samples += (label == pred).long().sum().item()
            # Backpropagate the loss to compute the gradients.
            loss.backward()
            # Clip the gradients to avoid exploding gradients.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            # Update the model parameters based on the computed gradients.
            optimizer.step()
            # Every 10 batches (except for the first batch), perform the following steps:
            if i % 10 == 0 and i > 0:
                # Calculate the average loss for the epoch so far.
                train_loss = total_loss / total_samples
                #  Calculate the accuracy for the epoch so far.
                train_accuracy = correct_samples / total_samples
                # Reset the total loss for the epoch to 0.
                # Reset the total number of samples seen in the epoch to 0.
                # Reset the total number of correct predictions in the epoch to 0.
                total_loss = 0
                total_samples = 0
                correct_samples = 0
                # Perform validation on the validation dataset and get the validation accuracy and loss.
                val_accuracy, val_loss = eval(model, val_loader, criterion)
                # Print out the training and validation loss and accuracy for the current batch.
                print('[epoch %2d] [step %3d] train_loss: %.4f train_acc: %.4f val_loss: %.4f val_acc: %.4f'
                      % (epoch, i, train_loss, train_accuracy, val_loss, val_accuracy))
                if val_accuracy > max_val_accuracy:
                    max_val_accuracy = val_accuracy
                    # torch.save(aspect_term_model.state_dict(), model_path)
                if val_loss < min_val_loss:
                    min_val_loss = val_loss
                    if epoch > 0:
                        torch.save(model.state_dict(), model_path)
        end = time.time()
        print('time: %.4fs' % (end - start))
    print('max_val_accuracy:', max_val_accuracy)


    Now Trying To Train The Model Using CUDNN Backend Allowing GPU Based Operation

In [8]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

train(config)

[epoch  0] [step  10] train_loss: 0.3580 train_acc: 0.4190 val_loss: 0.3715 val_acc: 0.4535
[epoch  0] [step  20] train_loss: 0.3404 train_acc: 0.4437 val_loss: 0.3478 val_acc: 0.4535
[epoch  0] [step  30] train_loss: 0.3323 train_acc: 0.4547 val_loss: 0.3250 val_acc: 0.5480
[epoch  0] [step  40] train_loss: 0.3059 train_acc: 0.5750 val_loss: 0.3189 val_acc: 0.5398
[epoch  0] [step  50] train_loss: 0.3014 train_acc: 0.5500 val_loss: 0.3099 val_acc: 0.5556
[epoch  0] [step  60] train_loss: 0.2872 train_acc: 0.5609 val_loss: 0.2888 val_acc: 0.5863
[epoch  0] [step  70] train_loss: 0.2814 train_acc: 0.5797 val_loss: 0.2816 val_acc: 0.5916
[epoch  0] [step  80] train_loss: 0.2737 train_acc: 0.5969 val_loss: 0.3002 val_acc: 0.5856
[epoch  0] [step  90] train_loss: 0.2823 train_acc: 0.5531 val_loss: 0.2772 val_acc: 0.6171
[epoch  0] [step 100] train_loss: 0.2692 train_acc: 0.6062 val_loss: 0.2824 val_acc: 0.6119
[epoch  0] [step 110] train_loss: 0.2612 train_acc: 0.6141 val_loss: 0.2889 val_

    Now For The Testing Of This Model

In [9]:
import torch
import os
from train import make_aspect_term_model, make_aspect_category_model
from train.make_data import make_term_test_data, make_category_test_data
from train.eval import eval

def test(config):
    # Pull The Config From The  Config File
    mode = config['mode']
    # Based On The Config Make The Model
    if mode == 'term':
        model = make_aspect_term_model.make_model(config)
    else:
        model = make_aspect_category_model.make_model(config)
    # Move The model to the GPU If possible
    model = model.cuda()
    # Get the saved model's path
    model_path = os.path.join(config['base_path'], 'checkpoints/%s.pth' % config['aspect_' + mode + '_model']['type'])
    # Load It into the torch Direcotry
    model.load_state_dict(torch.load(model_path))
    # In case of Term Data We check The Term Test.xml or the Category.xml for Category
    if mode == 'term':
        test_loader = make_term_test_data(config)
    else:
        test_loader = make_category_test_data(config)
    
    # Call The Custom Evaluation Method To Load the method 
    test_accuracy = eval(model, test_loader)
    # Print The Accuracy Relative to the test data.
    print('test:\taccuracy: %.4f' % (test_accuracy))

test(config)

test:	accuracy: 0.7455


##### To Test Our Model We Need To Process Our Sentence into Word2Vec and Index2Word Then Pass The Vector Values Into The Neural Network

To Check Custom Inputs -> Output We need to write custom Functions